# Import everything


In [1]:
from typing import Tuple
import torch
import numpy as np
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader
from settings import cfg
from helper import boilerplate, debug, loader


Initialized


# Get the dataset

In [2]:
dataset_image, dataset_label = loader.load_data(cfg.train_paths, mmap_mode='c')
punching = dataset_label.sum()
not_punching = dataset_label.shape[0] - punching
print(f'''Stats:
    | Number of punching: {punching}
    | Number of not-punching: {not_punching}''')


Stats:
    | Number of punching: 6382
    | Number of not-punching: 34766


# Boilerplate Code

In [3]:
Data = boilerplate.TrainingDataset


# Model

In [4]:
from settings.cfg import device, model, transforms, optimizer, idx_gen, batch_size

criterion = cfg.get_loss([not_punching, punching])
debug.print_model_size(model)


Param size: 3.670MB
Buffer size: 0.000MB


# Train

In [5]:
from sklearn.metrics import f1_score


def train(train_idx: np.ndarray) -> 'Tuple(float, float)':
    model.train()
    train = Data(dataset_image, dataset_label, train_idx)
    train_dataloader = DataLoader(train, batch_size=batch_size)
    total_loss_train = 0
    
    prediction_array = []
    label_array = []
    for image, label in tqdm(train_dataloader):
        image = image.to(device, dtype=torch.float)
        label = label.to(device, dtype=torch.uint8)

        image = transforms(image)
        # debug.print_image(image)
        output = model(image)

        batch_loss = criterion(output, label)
        total_loss_train += batch_loss.item()

        prediction = output.argmax(dim=1)

        prediction_array.append(prediction.cpu().numpy())
        label_array.append(label.cpu().numpy())

        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
        model.zero_grad()

    prediction_array = np.concatenate(prediction_array)
    label_array = np.concatenate(label_array)

    total_accumulate_train = (prediction_array == label_array).sum().item()
    f1_score_train = f1_score(label_array, prediction_array, average='macro')

    return total_loss_train, total_accumulate_train, f1_score_train


In [6]:
def judge(judge_idx: np.ndarray) -> 'Tuple(float, float)':
    model.eval()
    judge = Data(dataset_image, dataset_label, judge_idx)
    judge_dataloader = DataLoader(judge, batch_size=batch_size)
    total_loss_judge = 0

    prediction_array = []
    label_array = []
    with torch.no_grad():
        for image, label in tqdm(judge_dataloader):
            image = image.to(device, dtype=torch.float)
            label = label.to(device, dtype=torch.uint8)

            output = model(image)
            batch_loss = criterion(output, label)
            total_loss_judge += batch_loss.item()

            prediction = output.argmax(dim=1)
            prediction_array.append(prediction.cpu().numpy())
            label_array.append(label.cpu().numpy())

    prediction_array = np.concatenate(prediction_array)
    label_array = np.concatenate(label_array)

    total_accumulate_judge = (prediction_array == label_array).sum().item()
    f1_score_judge = f1_score(label_array, prediction_array, average='macro')

    return total_loss_judge, total_accumulate_judge, f1_score_judge

In [7]:
min_judge_loss = float('inf')


for epoch, (train_idx, judge_idx) in enumerate(idx_gen.split(dataset_label)):
    print(f'''Starting epoch {epoch+1}
        | Train size: {train_idx.shape[0]}
        | Judge size: {judge_idx.shape[0]}''')
    total_loss_train, total_accumulate_train, f1_score_train = train(train_idx)
    total_loss_judge, total_accumulate_judge, f1_score_judge = judge(judge_idx)

    print(
        f'''Epoch: {epoch+1} 
        | Train Loss: {total_loss_train / len(train_idx):.3f}
        | Train Accuracy: {total_accumulate_train/len(train_idx):.3f}
        | Train F1 Score: {f1_score_train:.3f}
        | Val Loss: {total_loss_judge/len(judge_idx):.3f}
        | Val Accuracy: {total_accumulate_judge/len(judge_idx):.3f}
        | Val F1 Score: {f1_score_judge:.3f}''')

    if min_judge_loss > total_loss_judge/len(judge_idx):
        min_judge_loss = total_loss_judge/len(judge_idx)
        torch.save(model.state_dict(), cfg.model_path)
        print(f"Save model because val loss improve loss {min_judge_loss:.3f}")


Starting epoch 1
        | Train size: 37033
        | Judge size: 4115


  0%|          | 0/1158 [00:00<?, ?it/s]

  0%|          | 0/129 [00:00<?, ?it/s]

Epoch: 1 
        | Train Loss: 0.022
        | Train Accuracy: 0.789
        | Train F1 Score: 0.491
        | Val Loss: 0.022
        | Val Accuracy: 0.845
        | Val F1 Score: 0.458
Save model because val loss improve loss 0.022
Starting epoch 2
        | Train size: 37033
        | Judge size: 4115


  0%|          | 0/1158 [00:00<?, ?it/s]

...

  0%|          | 0/129 [00:00<?, ?it/s]

Epoch: 19 
        | Train Loss: 0.022
        | Train Accuracy: 0.844
        | Train F1 Score: 0.458
        | Val Loss: 0.022
        | Val Accuracy: 0.849
        | Val F1 Score: 0.459
Starting epoch 20
        | Train size: 37034
        | Judge size: 4114


  0%|          | 0/1158 [00:00<?, ?it/s]

  0%|          | 0/129 [00:00<?, ?it/s]

Epoch: 20 
        | Train Loss: 0.022
        | Train Accuracy: 0.845
        | Train F1 Score: 0.458
        | Val Loss: 0.022
        | Val Accuracy: 0.847
        | Val F1 Score: 0.459
